In [333]:
import glob
import os
import re
import itertools
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits

In [334]:
def flatten(listOfLists):
    "Flatten one level of nesting"
    return itertools.chain.from_iterable(listOfLists)

In [335]:
datadir = '../data/combine/'
listinfo = '../data/infofolder/'
lista = glob.glob(listinfo+'*.txt')
ralista = []
declista = []
fulllist = []
dategrism = []
dateradec =[]
for f in lista:
    #g = f.split('.')[0][-6:]
    g =re.search(r'grism\d',f)[0]
    with open(f,'r') as file:
        templines = []
        templines.append(g)
        for lines in file:
            templines.append(lines.strip())
        rapoint = float(templines[4].split(' ')[1])
        decpoint = float(templines[6].split(' ')[1])
        c_icrs = SkyCoord(ra=rapoint*u.degree, dec=decpoint*u.degree, frame='icrs')
        coord = c_icrs.to_string('hmsdms').split(' ')
        rapoint2 = coord[0]
        decpoint2 = coord[1]

        raifu = templines[-2].split(' ')[1]
        decifu = templines[-1].split(' ')[1]
        date = templines[2].split( )[1]
        date2 = re.search('(\d+)\-(\d+)\-(\d+)T(\d+)\:(\d+)\:(\d+)',date)[0]
        dategrism.append((g,date2))
        fulllist.append(templines)
        dateradec.append((date2,raifu,decifu,rapoint2,decpoint2))
    ralista.append(ra)
    declista.append(dec)


In [ ]:
RA

In [226]:
dateg = np.array(dategrism)
dividedlist = []
for gr in set(dateg[:,0]):
    t= np.where(dateg[:,0] == gr)
    lista = dateg[t,1]
    dividedlist.append(lista.flatten())


In [249]:
dateg[:,1]

array(['2003-06-04T06:33:31', '2003-06-06T06:24:11', '2003-06-04T06:01:36',
       '2003-06-22T02:19:13', '2003-06-06T06:52:40', '2003-06-22T03:14:21',
       '2003-06-24T04:24:10', '2003-06-22T02:40:57', '2003-06-02T03:08:37',
       '2003-06-02T03:22:58', '2003-06-02T03:37:12', '2003-06-03T08:40:04',
       '2003-06-03T08:54:19', '2003-06-03T09:08:34', '2003-06-04T03:46:44',
       '2003-06-04T04:00:58', '2003-06-04T04:15:12', '2003-06-04T04:40:29',
       '2003-06-04T05:16:43', '2003-06-04T05:39:52', '2003-06-06T04:43:29',
       '2003-06-06T06:38:25', '2003-06-22T03:36:05', '2003-06-22T07:03:15',
       '2003-06-22T06:41:25', '2003-06-24T04:55:50', '2003-06-24T05:17:34',
       '2003-06-25T02:14:35', '2003-06-24T05:47:09', '2003-06-25T02:41:33',
       '2003-06-24T06:08:54', '2003-06-25T03:03:17', '2003-06-04T06:33:01',
       '2003-06-04T04:39:59', '2003-06-24T04:24:10', '2003-06-06T06:52:19',
       '2003-06-02T03:08:10', '2003-06-02T03:22:30', '2003-06-02T03:36:45',
       '2003

In [310]:
datecombine = '2003-06-24T04:55:50'
newd = '{}{}/'.format(datadir,datecombine)
print(newd)

../data/combine/2003-06-24T04:55:50/


In [311]:
if not os.path.exists(newd):
    os.mkdir(newd)
    #os.system('cp {} {}'.format(ob[0],pathob))

In [330]:
inputfiles = []
infofile = "{}{}".format(newd,'info.txt')
open(infofile, 'w').close()


for i in dategrism:
    if i[1] == datecombine:
        newname = '{}{}-{}.fits'.format(newd,i[0],i[1])
        fittocombine  = '../data/{}/objects/{}/ifu_fov.fits'.format(i[0],i[1])
        temp = fits.open(fittocombine)
        
        with open(infofile,'a') as f:
            #temp = fits.open(ob[0])
            fitheader = temp[0].header
            date = fitheader['DATE-OBS']
            exptime = fitheader['EXPTIME']
            ra = fitheader['RA']
            dec = fitheader['DEC']
            c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
            c2 = c.to_string('hmsdms').split(' ')
            ra2 = ifucoord[0]
            dec2 = ifucoord[1]
            ifura = fitheader['HIERARCH ESO INS IFU RA']
            ifudec = fitheader['HIERARCH ESO INS IFU DEC']
            c_icrs = SkyCoord(ra=ifura*u.degree, dec=ifudec*u.degree, frame='icrs')
            ifucoord = c_icrs.to_string('hmsdms').split(' ')
            ifura2 = ifucoord[0]
            ifudec2 = ifucoord[1]
            name = fitheader['HIERARCH ESO OBS TARG NAME']
            quad  = fitheader['HIERARCH ESO OCS CON QUAD']
            f.write('\nquad: {} \nname: {} \ndate: {}\nexptime: {}\nra: {} \nra2 {} \ndec: {} \ndec2 {} \nifuRA {} \nifuDEC {}\n'.format(quad,name,date,exptime,ra,ra2,dec,dec2,ifura2,ifudec2))

        command = 'cp {} {}'.format(fittocombine,newname)
        inputfiles.append([(newname,'IFU_FOV')])
        os.system(command)
inputfiles = list(flatten(inputfiles))


In [331]:
temp[0].header

SIMPLE  =                    T / # of pixels along Y                            
BITPIX  =                  -32 / No. of bits per pixel                          
NAXIS   =                    2 / # of axes in frame                             
NAXIS1  =                   80 / Pixel in X                                     
NAXIS2  =                   80 / Pixel in Y                                     
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
ORIGIN  = 'ESO     '           / European Southern Observatory                  
DATE    = '2018-07-10T22:51:43' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
MJD-OBS =       52814.20544635 / Obs start 2003-06-24T04:55:50.565              
DATE-OBS= '2003-06-24T04:55:50.564' / Date of observation                       
EXPTIME =                125

In [323]:
inputfiles

[('../data/combine/2003-06-24T04:55:50/grism1-2003-06-24T04:55:50.fits',
  'IFU_FOV'),
 ('../data/combine/2003-06-24T04:55:50/grism2-2003-06-24T04:55:50.fits',
  'IFU_FOV'),
 ('../data/combine/2003-06-24T04:55:50/grism3-2003-06-24T04:55:50.fits',
  'IFU_FOV'),
 ('../data/combine/2003-06-24T04:55:50/grism4-2003-06-24T04:55:50.fits',
  'IFU_FOV')]

In [243]:
filenamescombinefov = newd+'ifucombinefov.sof'

with open(filenamescombinefov,'w') as file:
    for sof in inputfiles:
        string = '{} {}\n'.format(sof[0],sof[1])
        file.write(string)

In [244]:
esorexpath = '../esorex/bin/esorex'
outputdir = '--output-dir={}'.format(newd)
logdir = '--log-dir={}'.format(newd)
routine = 'vmifucombine'
command = '{} {} {} {} {} '.format(esorexpath,outputdir,logdir,routine, filenamescombinefov)
print(command)
os.system(command)

../esorex/bin/esorex --output-dir=../data/combine/2003-06-24T04:55:50/ --log-dir=../data/combine/2003-06-24T04:55:50/ vmifucombine ../data/combine/2003-06-24T04:55:50/ifucombinefov.sof 


0

# Now all

In [271]:
for d in dateg[:,1]:
    datecombine = d
    newd = '{}{}/'.format(datadir,datecombine)
    if not os.path.exists(newd):
        os.mkdir(newd)
    inputfiles = []
    for i in dategrism:
        if i[1] == datecombine:
            newname = '{}{}-{}.fits'.format(newd,i[0],i[1])
            fittocombine  = '../data/{}/objects/{}/ifu_fov.fits'.format(i[0],i[1])
            command = 'cp {} {}'.format(fittocombine,newname)
            inputfiles.append([(newname,'IFU_FOV')])
            os.system(command)
    inputfiles = list(flatten(inputfiles))
    filenamescombinefov = newd+'ifucombinefov.sof'

    with open(filenamescombinefov,'w') as file:
        for sof in inputfiles:
            string = '{} {}\n'.format(sof[0],sof[1])
            file.write(string)
    esorexpath = '../esorex/bin/esorex'
    outputdir = '--output-dir={}'.format(newd)
    logdir = '--log-dir={}'.format(newd)
    routine = 'vmifucombine'
    command = '{} {} {} {} {} '.format(esorexpath,outputdir,logdir,routine, filenamescombinefov)
    os.system(command)

    inputfilescube = []
    for i in dategrism:
        if i[1] == datecombine:
            newnamescience = '{}science{}-{}.fits'.format(newd,i[0],i[1])
            fittocombine  = '../data/{}/objects/{}/ifu_science_reduced.fits'.format(i[0],i[1])
            command = 'cp {} {}'.format(fittocombine,newnamescience)
            inputfilescube.append([(newnamescience,'IFU_SCIENCE_REDUCED')])
            os.system(command)
    inputfilescube = list(flatten(inputfilescube))

    filenamecube = newd+'cubecombine.sof'
    with open(filenamecube,'w') as file:
        for sof in inputfilescube:
            string = '{} {}\n'.format(sof[0],sof[1])
            file.write(string)
    routine = 'vmifucombinecube'
    commandcube = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamecube)
    os.system(commandcube)


In [299]:
dateradec

[('2003-06-04T06:33:31',
  '18h35m43.8111s',
  '-32d59m29.456s',
  '18h36m36.6677s',
  '-32d59m31.056s'),
 ('2003-06-06T06:24:11',
  '18h35m43.8108s',
  '-32d59m29.456s',
  '18h36m36.6674s',
  '-32d59m31.056s'),
 ('2003-06-04T06:01:36',
  '18h35m44.7009s',
  '-32d59m41.66s',
  '18h36m37.5595s',
  '-32d59m43.26s'),
 ('2003-06-22T02:19:13',
  '18h35m44.7002s',
  '-32d59m19.664s',
  '18h36m37.5552s',
  '-32d59m21.264s'),
 ('2003-06-06T06:52:40',
  '18h35m43.8108s',
  '-32d59m29.456s',
  '18h36m36.6674s',
  '-32d59m31.056s'),
 ('2003-06-22T03:14:21',
  '18h35m44.6994s',
  '-32d59m41.66s',
  '18h36m37.5581s',
  '-32d59m43.26s'),
 ('2003-06-24T04:24:10',
  '18h35m44.6994s',
  '-32d59m41.66s',
  '18h36m37.5581s',
  '-32d59m43.26s'),
 ('2003-06-22T02:40:57',
  '18h35m44.7002s',
  '-32d59m19.664s',
  '18h36m37.5552s',
  '-32d59m21.264s'),
 ('2003-06-02T03:08:37',
  '18h35m43.8111s',
  '-32d59m29.456s',
  '18h36m36.6677s',
  '-32d59m31.056s'),
 ('2003-06-02T03:22:58',
  '18h35m43.8111s',
  '-32d

In [288]:
5-4

1

In [291]:
templines

['grism4',
 'name: NGC-6652-B',
 'date: 2003-06-25T03:03:17.100',
 'exptime: 1249.996',
 'ra: 279.161062',
 'ra2 18:36:38.6',
 'dec: -32.99001',
 'dec2 32:59:24.0',
 'ifuRA 18h35m45.7994s',
 'ifuDEC -32d59m22.4324s']

In [279]:
/re/a

SyntaxError: invalid syntax (<ipython-input-279-423246f35b9c>, line 1)